# Nullable Reference Types
* In the below code `MiddleName[0]` can result in null reference exception. 
* To guard against it, we can use package ReSharper Annotations which can provide annotations like `[CanBeNull]` which can be put to tell static code analyzer to tell this can be null.
* In C# 8, We have to opt into this feature, this can be done @ class level by putting a directive `#nullable enable`, this makes c# all property non nullable which will cause warning.
* in Compilation it would not make `String?` as `Nullable<String>` as `NUllable<T>` requires a struct or value type so this ? is only for compiler and static code analyzes tool when during final build `String? Middlename` --> `String MiddleName`;

<br/>\

* Overriding Null Checks:
    * way1: Make type not nullable by not putting ?
    * Way2: Write expression with Bang(!):
        ```csharp
        (null as Person)!.FirstName; // No warning will come due to bang operator
        (null as Person)!!!!!!!!!!!!.FirstName; // Also legal code
        (null as Person).FirstName; // Compiler understands it is null and gives a warning.
        (null as Person)!?.FirstName; //will do null check
       //SHould not compile (null as Person)?!.FirstName; 
        ```

In [5]:
# nullable enable 
public class Person {
    public string FirstName {get; set; }
    public string? MiddleName {get; set; }
    public string LastName {get; set; }

    public Person(string firstName , string? middleName , string lastName ) => (FirstName, MiddleName, LastName) = (firstName, middleName, lastName);

    // This warning will come due to we have made middleName nullable type. 
    public string FullName => $"{FirstName} {MiddleName[0]} {LastName}";
    // If middleName is null then MiddleName?[0] is null and in string it will become empty string.
    public string FullName1 => $"{FirstName} {MiddleName?[0]} {LastName}";

}

# Indices and Ranges
* Two new types:
    1. Index refers to single position
        * Points into some array or similar types which can be indexed. 
        * It has `Value` and `IsFromEnd` properties
    2. Range refers to range of positions
        * It is 2 indices
        * Syntax = `X..Y` from index X to Y.

In [23]:

        //Indexes
        Index i0 = 0;
        Index i1 = new Index(1, false);
        var i2 = ^0; // Index(0, true)

        var items = new [] {1, 2, 3, 4, 5, 6, 7, 8, 9, 0};
        var zero = items[^1];
        Console.WriteLine(zero);
        items[^1] = 10;
        var arr = items[0..4]; // {1,2,3,4} end it is not included.
        var arr1 = items[0..^1]; // {1,2,3,4,5} end is included.

0


# Default Interface Members
* Non abstract methods can be added with implementation in interfaces.
* Due to this:
    1. Interface syntax extended to accept protected, internal, public and virtual.
    2. default interface methods are virtual unless explicitly marked sealed/private.
    3. Interface without bodies are abstract
    4. interface can implement interfaces.



In [13]:
public interface IHuman {
    string Name {get; set;}

    void SayHello(){
        Console.WriteLine($"Hello my name is {Name}");
    }
}

public interface IFriendlyHuman: IHuman{
    void SayHello(){
        Console.WriteLine($"Greetings my name is {Name}");
    }
}

public class Human: IFriendlyHuman{
    public string Name {get; set;}
}

    ((IHuman)new Human()).SayHello(); // Hello my name is abhishek
    ((IFriendlyHuman)new Human()).SayHello(); // Greetings my name is abhishek 

    var human = new Human();
    //human.SayHello(); On class object the default implemented method is not available it is available only on variables casted as that interface type.


Hello my name is 
Greetings my name is 


# Diamond Inheritance Problem with default interface method
* IN below code this error will come: `Interface member 'ITalk.Greet()' does not have a most specific implementation. Neither 'IAmIndian.ITalk.Greet()', nor 'IAmAmerican.ITalk.Greet()' are most specific.`

In [ ]:
public interface ITalk{
    string Greet();
}

interface IAmIndian: ITalk{
    string ITalk.Greet() => "Namaste";
}

interface IAmAmerican: ITalk {
    string ITalk.Greet() => "Howdy";
}

class DualNational : IAmIndian, IAmAmerican{
    
}
 

# Pattern Matching
* `Property Patterns Matching` were introduced as shown in PhoneNumber example.
* `Recursive Pattern Matching` were introduced so we can go in nested object and match `{PhoneNumber: {Code: 91}} => "India` . Here we can have `variable` declarations as well. `{Name: var name} => $"No Idea where {name} lives"`

## Switch based validations 
```csharp
var error = person switch {
    null => "object Missing",
    {PhoneNumber: null} => "Phone number is missing entirely", 
    {PhoneNumber: {Number: 0}} => "Number is missing",
    {PhoneNumber: {Code: var code}} => when code < 0 => "WTF?",
    {} => null //no error
};

    if(error != null)
        throw new ArgumentException(error);
```

## Recursive Type Check with Pattern Matching
```csharp
IEnumerable<int> GetMainOfficeNumbers(){
    foreach(Var ph in numbers){
        if(ph is ExtendedPhoneNumber {Office: 'main'}) // type and value matching
            yield return ph.Number;
    }
}
```

In [26]:
#nullable disable
public struct PhoneNumber {
    public int Code, Number;
}

var phoneNumber = new PhoneNumber(){
    Code = 91,
    Number = 12345332
};

// if nothing is match then c# would throw an exception. Switch expression exception.
var origin = phoneNumber switch {
    {Number: 112} => "Emergency",
    {Code: 91} => "India"
};

Console.WriteLine(origin);
// fix for warning
var origin1 = phoneNumber switch {
    {Number: 112} => "Emergency",
    {Code: 91} => "India",
    {} => "Intermediate",// phone number is not null but doesn't match any case
};

Error: (21,5): error CS8510: The pattern is unreachable. It has already been handled by a previous arm of the switch expression or it is impossible to match.
(22,5): error CS8510: The pattern is unreachable. It has already been handled by a previous arm of the switch expression or it is impossible to match.
(23,5): error CS8510: The pattern is unreachable. It has already been handled by a previous arm of the switch expression or it is impossible to match.